In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import decimate
from gait_events import gait_events_HC_JA
from dtaidistance import dtw
from tqdm import tqdm
from tslearn.metrics import dtw
import json

/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [12]:
# 3. DTW Iterate over patients with a progress bar
# Top‐level bar over patients
dtw_all = {}
for pid in tqdm(patient_ids, desc='Patients', position=0):
    tqdm.write(f"Processing patient {pid}...")
    folder = os.path.join(base_folder, pid)
    dfs, files = load_patient_data(folder, pid)
    if not dfs:
        tqdm.write(f"  No data for {pid}, skipping.")
        continue

    # 1) Representative cycle per trial
    rep_cycles, trial_names = [], []

    # Bar over trials
    for df, fname in tqdm(zip(dfs, files),
                          total=len(dfs),
                          desc=f'Trials {pid}',
                          position=1,
                          leave=False):
        cycles = segment_downsamp(df)
        for idx1, c1 in enumerate(cycles):
            for idx2, c2 in enumerate(cycles):
                if not np.array_equal(c1, c2):
                    dtw_score = dtw(c1, c2)
                    if pid not in dtw_all:
                        dtw_all[pid] = {}
                    if f'{idx1}|{idx2}' not in dtw_all[pid]:
                        dtw_all[pid][f'{idx1}|{idx2}'] = []
                    dtw_all[pid][f'{idx1}|{idx2}'].append(dtw_score)

Patients:   0%|          | 0/1 [00:00<?, ?it/s]

Processing patient S002...


Skipping cycle of length 21 (too short for downsampling).


Skipping cycle of length 24 (too short for downsampling).


Skipping cycle of length 26 (too short for downsampling).


Patients: 100%|██████████| 1/1 [01:42<00:00, 102.74s/it]


In [3]:
json.dump(dtw_all, open(os.path.join(output_base, 'dtw_all.json')))

NameError: name 'output_base' is not defined

In [4]:
file_path = 'DTW_downsampled2/dtw_all.json'
with open(file_path, 'r') as f:
    dtw_all = json.load(f)

with open(file_path, 'w') as f:
    json.dump(dtw_all, f)  

In [ ]:


# Cargar los resultados desde el archivo JSON
with open("DTW_downsampled2/dtw_all.json", "r") as f:
    dtw_all = json.load(f)

# Función para calcular la media de DTW por trial (usando un ciclo representativo)
def calculate_representative_cycle_dtw(patient_dtw):
    # Crear un dataframe para almacenar los resultados DTW entre trials
    trials = list(patient_dtw.keys())  # Ejemplo: ["0|1", "0|2", "0|3"]
    
    dtw_values = []
    
    # Iterar por cada par de trials
    for trial_1 in trials:
        for trial_2 in trials:
            if trial_1 != trial_2:
                # Extraer las distancias DTW de la comparación
                dtw_values.append(np.mean(patient_dtw[trial_1]))  # Usamos la media de los DTW entre ciclos del trial
    
    return np.mean(dtw_values)

# Análisis para cada paciente
def analyze_patient_dtw(patient_id, dtw_all):
    patient_dtw = dtw_all.get(patient_id, {})
    
    if not patient_dtw:
        print(f"No data for patient {patient_id}")
        return
    
    print(f"Analyzing patient {patient_id}...")

    # Calcular las distancias DTW promedio entre todos los trials para el paciente
    dtw_values = []
    for trial_1 in patient_dtw.keys():
        for trial_2 in patient_dtw.keys():
            if trial_1 != trial_2:
                # Tomamos la distancia media entre ciclos comparados para cada trial
                dtw_values.append(np.mean(patient_dtw[trial_1]))  # Usamos la media de DTW entre ciclos
    
    # Crear un dataframe de los resultados
    df_dtw = pd.DataFrame(dtw_values, columns=['DTW'])
    return df_dtw

# Visualización: crear un gráfico de barras por paciente
def plot_dtw_analysis(patient_dtw_df, patient_id):
    if patient_dtw_df is not None:
        plt.figure(figsize=(10, 6))
        plt.hist(patient_dtw_df['DTW'], bins=20, color='skyblue', edgecolor='black')
        plt.title(f"DTW Distribution for Patient {patient_id}")
        plt.xlabel('DTW Distance')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.show()

# Función principal para iterar sobre todos los pacientes
def analyze_all_patients(dtw_all):
    for patient_id in dtw_all.keys():
        # Analizamos el DTW por paciente
        patient_dtw_df = analyze_patient_dtw(patient_id, dtw_all)
        
        # Graficamos el análisis
        plot_dtw_analysis(patient_dtw_df, patient_id)

# Llamamos la función para analizar todos los pacientes
analyze_all_patients(dtw_all)


Analyzing patient S002...
